<a href="https://colab.research.google.com/github/lialic/computer_vision/blob/master/Midterm/VGGNet_on_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mount drive and import neccessry libs

In [ ]:
import numpy as np
import pandas as pd

from tensorflow.python.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D 
from tensorflow.keras.utils import to_categorical
from tensorflow.python.keras.preprocessing.image import array_to_img
from tensorflow.python.keras.models import Sequential, load_model
from keras.optimizers import adam_v2 as Adam
from tensorflow.python.keras.callbacks import TensorBoard, ModelCheckpoint

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Unarchive data in the environment

In [ ]:
import shutil
shutil.unarchive('/cotent/drive/MyDrive/Data/mnist_png.tar.gz', '/content/mnist')

In [ ]:
path = './mnist/mnist_png/training'

image_path = []

# Adding all 10 classes picture name into 
for i in range(10):
    dir = os.path.join(path, str(i))
    for file in os.listdir(dir):
        image_path.append(os.path.join(dir, file))

# VGGNet

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
import seaborn as sns

In [ ]:
datagen = ImageDataGenerator()
generator = datagen.flow_from_directory(
    path,
    target_size=(224, 224), batch_size=128, class_mode = 'sparse'
)

Found 43746 images belonging to 10 classes.


In [ ]:
from tensorflow.keras.applications import VGG16
model = VGG16()

553476096/553467096 [==============================] - 5s 0us/step


In [ ]:
model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [ ]:
from keras import backend as K
K.learning_phase()

0

In [ ]:
from tensorflow.keras.models import Model
a = Model(inputs=model.input, outputs=model.get_layer('fc2').output)

In [ ]:
a.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [ ]:
image = np.zeros((1,4096))
label = np.zeros((1))

for i, (inputs_batch, labels_batch) in enumerate(generator):
    features_batch = a.predict(inputs_batch)
    image = np.concatenate((image, features_batch), axis = 0)
    label = np.concatenate((label, labels_batch), axis = 0)

    if i * generator.batch_size >= generator.samples:
        break

KeyboardInterrupt: ignored

In [ ]:
image1 = image[1:1160,:]
label1 = label[1:1160]

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(image1, label1, test_size=0.33, random_state=42)

In [ ]:
from sklearn import svm
clf = svm.SVC()
clf.fit(X_train, y_train)

In [ ]:
y_predict = clf.predict(X_test)

In [ ]:
class_names = np.arange(10)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_test, y_predict, target_names=class_names))
print( confusion_matrix(y_test, y_predict))
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test,y_predict))